In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import requests
from newspaper import Article

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}

article_url = "https://www.artificialintelligence-news.com/2022/01/25/meta-claims-new-ai-supercomputer-will-set-records/"

session = requests.Session()

try:
    response = session.get(article_url, headers=headers, timeout=10)

    if response.status_code == 200:
        article = Article(article_url)
        article.download()
        article.parse()

        print(f"Title: {article.title}")
        print(f"Text: {article.text}")
    else:
        print(f"Failed to fetch article at {article_url}")
except Exception as e:
    print(f"Error occurred while fetching article at {article_url}: {e}")

Title: Meta claims its new AI supercomputer will set records
Text: Meta (formerly Facebook) has unveiled an AI supercomputer that it claims will be the world’s fastest.

The supercomputer is called the AI Research SuperCluster (RSC) and is yet to be fully complete. However, Meta’s researchers have already begun using it for training large natural language processing (NLP) and computer vision models.

RSC is set to be fully built in mid-2022. Meta says that it will be the fastest in the world once complete and the aim is for it to be capable of training models with trillions of parameters.

“We hope RSC will help us build entirely new AI systems that can, for example, power real-time voice translations to large groups of people, each speaking a different language, so they can seamlessly collaborate on a research project or play an AR game together,” wrote Meta in a blog post.

“Ultimately, the work done with RSC will pave the way toward building technologies for the next major computing

In [7]:
from langchain.schema import (
    HumanMessage
)

# we get the article data from the scraping part
article_title = article.title
article_text = article.text

# prepare template for prompt
template = """
As an advanced AI, you've been tasked to summarize online articles into bulleted points. Here are a few examples of how you've done this in the past:

Example 1:
Original Article: 'The Effects of Climate Change
Summary:
- Climate change is causing a rise in global temperatures.
- This leads to melting ice caps and rising sea levels.
- Resulting in more frequent and severe weather conditions.

Example 2:
Original Article: 'The Evolution of Artificial Intelligence
Summary:
- Artificial Intelligence (AI) has developed significantly over the past decade.
- AI is now used in multiple fields such as healthcare, finance, and transportation.
- The future of AI is promising but requires careful regulation.

Now, here's the article you need to summarize:

==================
Title: {article_title}

{article_text}
==================

Please provide a summarized version of the article in a bulleted list format.
"""

# format prompt
prompt = template.format(article_title=article.title, article_text=article.text)

messages = [HumanMessage(content=prompt)]
messages

[HumanMessage(content="\nAs an advanced AI, you've been tasked to summarize online articles into bulleted points. Here are a few examples of how you've done this in the past:\n\nExample 1:\nOriginal Article: 'The Effects of Climate Change\nSummary:\n- Climate change is causing a rise in global temperatures.\n- This leads to melting ice caps and rising sea levels.\n- Resulting in more frequent and severe weather conditions.\n\nExample 2:\nOriginal Article: 'The Evolution of Artificial Intelligence\nSummary:\n- Artificial Intelligence (AI) has developed significantly over the past decade.\n- AI is now used in multiple fields such as healthcare, finance, and transportation.\n- The future of AI is promising but requires careful regulation.\n\nNow, here's the article you need to summarize:\n\n==================\nTitle: Meta claims its new AI supercomputer will set records\n\nMeta (formerly Facebook) has unveiled an AI supercomputer that it claims will be the world’s fastest.\n\nThe supercom

In [8]:
from langchain.chat_models import ChatOpenAI

# load the model
chat = ChatOpenAI(model_name="gpt-4-turbo", temperature=0)

In [9]:
# generate summary
summary = chat(messages)
print(summary.content)

- Meta (formerly Facebook) has announced the development of the AI Research SuperCluster (RSC), which it claims will be the world's fastest AI supercomputer upon completion.
- The RSC is designed to train large natural language processing (NLP) and computer vision models and is expected to be fully operational by mid-2022.
- Meta aims for the RSC to handle models with trillions of parameters and enhance capabilities such as real-time voice translations for diverse language groups.
- The supercomputer is projected to be significantly faster than Meta's current systems, being 20x faster than their V100-based clusters and improving training times for large-scale NLP workflows by 3x.
- RSC incorporates enhanced security and privacy controls, allowing Meta to utilize real-world data from its production systems for training, particularly for tasks like identifying harmful content.
- Meta highlights the RSC's role in advancing AI technologies for the metaverse, where AI-driven applications wi

# =========

In [10]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import validator, BaseModel, Field
from typing import List

# create output parser class
class ArticleSummary(BaseModel):
    title: str = Field(description="Title of the article")
    summary: List[str] = Field(description="Bulleted list summary of the article")

    # validating whether the generated summary has at least three lines
    @validator('summary')
    def has_three_or_more_lines(cls, list_of_lines):
        if len(list_of_lines) < 3:
            raise ValueError("Generated summary has less than three bullet points!")
        return list_of_lines

# set up output parser
parser = PydanticOutputParser(pydantic_object=ArticleSummary)

In [11]:
from langchain.prompts import PromptTemplate


# create prompt template
# notice that we are specifying the "partial_variables" parameter
template = """
You are a very good assistant that summarizes online articles.

Here's the article you want to summarize.

==================
Title: {article_title}

{article_text}
==================

{format_instructions}
"""

prompt_template = PromptTemplate(
    template=template,
    input_variables=["article_title", "article_text"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)


In [12]:
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain

# instantiate model class
model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

chain = LLMChain(llm=model, prompt=prompt_template)

# Run the LLMChain to get the AI-generated answer
output = chain.run({"article_title": article_title, "article_text":article_text})

# Parse the output into the Pydantic model
parsed_output = parser.parse(output)
print(parsed_output)


title='Meta claims its new AI supercomputer will set records' summary=["Meta (formerly Facebook) has unveiled an AI supercomputer called the AI Research SuperCluster (RSC) that it claims will be the world's fastest.", 'RSC is set to be fully built in mid-2022 and will be capable of training models with trillions of parameters.', 'Meta aims to use RSC for tasks like real-time voice translations and building technologies for the metaverse.', "RSC is expected to be 20x faster than Meta's current clusters and will improve training times for large-scale NLP workflows.", 'Meta designed RSC with security and privacy controls to use real-world examples from its production systems for training AI models.', 'The performance, reliability, security, and privacy aspects of RSC are said to be tackled at a large scale by Meta.']


In [15]:
print(prompt_template.format(article_title=article_title, article_text=article_text))


You are a very good assistant that summarizes online articles.

Here's the article you want to summarize.

Title: Meta claims its new AI supercomputer will set records

Meta (formerly Facebook) has unveiled an AI supercomputer that it claims will be the world’s fastest.

The supercomputer is called the AI Research SuperCluster (RSC) and is yet to be fully complete. However, Meta’s researchers have already begun using it for training large natural language processing (NLP) and computer vision models.

RSC is set to be fully built in mid-2022. Meta says that it will be the fastest in the world once complete and the aim is for it to be capable of training models with trillions of parameters.

“We hope RSC will help us build entirely new AI systems that can, for example, power real-time voice translations to large groups of people, each speaking a different language, so they can seamlessly collaborate on a research project or play an AR game together,” wrote Meta in a blog post.

“Ultimat

In [17]:
parsed_output.summary

["Meta (formerly Facebook) has unveiled an AI supercomputer called the AI Research SuperCluster (RSC) that it claims will be the world's fastest.",
 'RSC is set to be fully built in mid-2022 and will be capable of training models with trillions of parameters.',
 'Meta aims to use RSC for tasks like real-time voice translations and building technologies for the metaverse.',
 "RSC is expected to be 20x faster than Meta's current clusters and will improve training times for large-scale NLP workflows.",
 'Meta designed RSC with security and privacy controls to use real-world examples from its production systems for training AI models.',
 'The performance, reliability, security, and privacy aspects of RSC are said to be tackled at a large scale by Meta.']